# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
import joblib

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [19]:
# Memuat dataset hasil clustering
file_path = 'Clustering_Customers_result.csv'
data = pd.read_csv(file_path)
print("Dataset berhasil dimuat!")
print(data.head())

Dataset berhasil dimuat!
   CustomerID  Gender  Age  Annual Income ($)  Spending Score (1-100)  \
0           1    Male   19              15000                      39   
1           2    Male   21              35000                      81   
2           3  Female   20              86000                       6   
3           4  Female   23              59000                      77   
4           5  Female   31              38000                      40   

      Profession  Work Experience  Family Size  data_content_scaled  \
0     Healthcare                1            4            -2.086675   
1       Engineer                3            3            -1.650205   
2       Engineer                1            1            -0.537207   
3         Lawyer                0            2            -1.126441   
4  Entertainment                2            6            -1.584734   

  content_category  Cluster  
0              Low        2  
1              Low        2  
2           Medium 

# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [20]:
# 3. Preprocessing Data
# Encode kolom kategorikal
categorical_columns = ['Gender', 'Profession', 'content_category']
encoder = LabelEncoder()

for col in categorical_columns:
    data[col] = encoder.fit_transform(data[col])

# Memilih Fitur dan Label
features = ['Age', 'Annual Income ($)', 'Spending Score (1-100)',
            'Work Experience', 'Family Size', 'data_content_scaled',
            'Gender', 'Profession', 'content_category']
X = data[features]
y = data['Cluster']

# Standarisasi Data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Membagi Data Menjadi Data Latih dan Data Uji
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Tampilkan Bentuk Data
print("Shape of X_train\t:", X_train.shape)
print("Shape of X_test\t\t:", X_test.shape)
print("Shape of y_train\t:", y_train.shape)
print("Shape of y_test\t\t:", y_test.shape)

Shape of X_train	: (1572, 9)
Shape of X_test		: (393, 9)
Shape of y_train	: (1572,)
Shape of y_test		: (393,)


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [21]:
# Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

# Logistic Regression
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)

Tulis narasi atau penjelasan algoritma yang Anda gunakan.

**1. Random Forest**
- Definisi: Algoritma ensemble yang menggunakan banyak pohon keputusan untuk klasifikasi.
- Cara Kerja: Membuat beberapa pohon dengan sampel acak dari data dan fitur, lalu menggabungkan hasilnya melalui voting mayoritas.
- Keunggulan: Tahan terhadap overfitting, efektif untuk data non-linear, dan mampu menangani noise serta outlier.

**2. Regresi Logistik**
- Definisi: Model statistik untuk memprediksi probabilitas kelas target dalam masalah klasifikasi.
- Cara Kerja: Menggunakan fungsi logit untuk menghitung probabilitas dan menetapkan ambang batas untuk klasifikasi.
- Keunggulan: Mudah diinterpretasikan, baik untuk hubungan linear antara fitur dan target.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [22]:
# Evaluasi Random Forest
print("Random Forest - Accuracy\t:", accuracy_score(y_test, y_pred_rf))
print("Random Forest - F1 Score\t:", f1_score(y_test, y_pred_rf, average='weighted'))

# Evaluasi Logistic Regression
print("Logistic Regression - Accuracy\t:", accuracy_score(y_test, y_pred_lr))
print("Logistic Regression - F1 Score\t:", f1_score(y_test, y_pred_lr, average='weighted'))

Random Forest - Accuracy	: 0.9974554707379135
Random Forest - F1 Score	: 0.9974544258253566
Logistic Regression - Accuracy	: 0.989821882951654
Logistic Regression - F1 Score	: 0.9898016145044622


## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [23]:
# Hyperparameter tuning untuk Random Forest
param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

grid_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=3, scoring='f1_weighted')
grid_rf.fit(X_train, y_train)
print("Parameter terbaik untuk Random Forest:", grid_rf.best_params_)

Parameter terbaik untuk Random Forest: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}


## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [24]:
# Menggunakan model terbaik dari GridSearchCV
best_rf_model = grid_rf.best_estimator_
y_pred_best_rf = best_rf_model.predict(X_test)

# Evaluasi ulang model Random Forest
print("Tuned Random Forest - Accuracy:", accuracy_score(y_test, y_pred_best_rf))
print("Tuned Random Forest - F1 Score:", f1_score(y_test, y_pred_best_rf, average='weighted'))

Tuned Random Forest - Accuracy: 0.9974554707379135
Tuned Random Forest - F1 Score: 0.9974544258253566


## **e. Analisis Hasil Evaluasi Model Klasifikasi**

In [25]:
# Confusion Matrix dan Classification Report untuk kedua model
print("\nConfusion Matrix - Random Forest")
print(confusion_matrix(y_test, y_pred_rf))
print("\nConfusion Matrix - Logistic Regression")
print(confusion_matrix(y_test, y_pred_lr))

# Analisis hasil dengan Classification Report
print("\nClassification Report - Random Forest")
print(classification_report(y_test, y_pred_rf))

print("\nClassification Report - Logistic Regression")
print(classification_report(y_test, y_pred_lr))


Confusion Matrix - Random Forest
[[145   0   0]
 [  0 136   0]
 [  0   1 111]]

Confusion Matrix - Logistic Regression
[[145   0   0]
 [  2 133   1]
 [  0   1 111]]

Classification Report - Random Forest
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       145
           1       0.99      1.00      1.00       136
           2       1.00      0.99      1.00       112

    accuracy                           1.00       393
   macro avg       1.00      1.00      1.00       393
weighted avg       1.00      1.00      1.00       393


Classification Report - Logistic Regression
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       145
           1       0.99      0.98      0.99       136
           2       0.99      0.99      0.99       112

    accuracy                           0.99       393
   macro avg       0.99      0.99      0.99       393
weighted avg       0.99      0.99      0.99

Berikut adalah **rekomendasi** tahapannya.

**1. Perbandingan Hasil Evaluasi Sebelum dan Sesudah Tuning**

Berdasarkan informasi yang Anda berikan sebelumnya, baik Random Forest maupun Logistic Regression sudah mencapai akurasi dan F1-Score yang sangat tinggi (mendekati 1.0) sebelum tuning.

Setelah tuning, peningkatan performa mungkin tidak signifikan karena model sudah mendekati performa optimal. Namun, kita tetap perlu menganalisis Confusion Matrix dan Classification Report untuk melihat detail performa pada setiap kelas.

**2. Identifikasi Kelemahan Model**

**Random Forest:**

- **Confusion Matrix:** Menunjukkan performa yang hampir sempurna. Hampir semua prediksi benar, dengan hanya 1 kesalahan pada kelas 2 (1 data dari kelas 2 diprediksi sebagai kelas 1).
- **Classification Report:** Precision, recall, dan F1-score sangat tinggi (mendekati 1.0) untuk semua kelas. Ini menunjukkan model sangat baik dalam mengklasifikasikan data.

**Logistic Regression:**

- **Confusion Matrix:** Ada beberapa kesalahan prediksi, terutama pada kelas 1:
 - 2 data dari kelas 1 diprediksi sebagai kelas 0.
 - 1 data dari kelas 1 diprediksi sebagai kelas 2.
 - 1 data dari kelas 2 diprediksi sebagai kelas 1.
- **Classification Report:** Meskipun akurasi masih tinggi (0.99), precision, recall, dan F1-score untuk kelas 1 sedikit lebih rendah dibandingkan Random Forest.

**Overfitting/Underfitting:**
- Kedua model, terutama Random Forest, menunjukkan performa yang sangat tinggi pada data uji. Hal ini berpotensi mengindikasikan overfitting, di mana model terlalu kompleks dan "menghafal" data latih sehingga tidak dapat menggeneralisasi dengan baik pada data baru.
- Untuk memastikan apakah terjadi overfitting, Anda perlu melakukan evaluasi dengan data yang lebih banyak dan beragam, serta menggunakan teknik cross-validation (seperti k-fold cross-validation).

**3. Rekomendasi Tindakan Lanjutan**
1. **Evaluasi dengan Data Lebih Banyak:** Kumpulkan data tambahan yang lebih representatif dan beragam untuk menguji model secara lebih komprehensif.
2. **Cross-Validation:** Gunakan teknik cross-validation untuk mendapatkan estimasi performa model yang lebih robust dan mendeteksi potensi overfitting.
3. **Perhatikan Kelas 1 (Logistic Regression):** Jika fokus Anda pada kelas 1, Anda dapat mencoba meningkatkan performa Logistic Regression dengan:
 - Menyeimbangkan data (jika ada ketidakseimbangan kelas).
 - Mencoba teknik feature engineering untuk membuat fitur baru yang lebih informatif.
4. **Analisis Error:** Telusuri data yang salah diprediksi (berdasarkan Confusion Matrix) untuk memahami pola kesalahan dan mencari cara untuk meningkatkan model.

